<a href="https://colab.research.google.com/github/priyakumari55/DEEP-LEARNING/blob/main/Movie_Review_Sentiment_Analysis_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords 
import string
from collections import Counter
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
train = pd.read_csv('train[1].tsv', sep="\t")
test = pd.read_csv('test[1].tsv', sep="\t")

In [7]:
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [8]:
print ("Cleaning the tweets...\n")
clean_data = []
for row in train.Phrase:
    clean_data.append(tweet_cleaner_updated(row))

Cleaning the tweets...



/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [9]:
clean_df = pd.DataFrame(clean_data,columns=['Phrase'])
clean_df['PhraseId'] = train.PhraseId
clean_df['SentenceId'] = train.SentenceId
clean_df['Sentiment'] = train.Sentiment
clean_df.to_csv('cleaned_train_data.csv',encoding='utf-8')

In [10]:
csv = 'cleaned_train_data.csv'
cleaned_train_data = pd.read_csv(csv,index_col=0)
cleaned_train_data

,Phrase,PhraseId,SentenceId,Sentiment
0,series of escapades demonstrating the adage th...,1,1,1.0
1,series of escapades demonstrating the adage th...,2,1,2.0
2,series,3,1,2.0
3,NaN,4,1,2.0
4,series,5,1,2.0
...,...,...,...,...
22138,dim witted pairing,22139,993,1.0
22139,dim witted pairing,22140,993,1.0
22140,dim witted,22141,993,1.0
22141,of teen speak and animal gibberish,22142,993,1.0


In [11]:
cleaned_train_data[cleaned_train_data.isnull().any(axis=1)]

,Phrase,PhraseId,SentenceId,Sentiment
3,NaN,4,1,2.0
37,NaN,38,1,2.0
62,NaN,63,1,2.0
92,NaN,93,3,2.0
98,NaN,99,3,2.0
...,...,...,...,...
18930,NaN,18931,831,2.0
19205,NaN,19206,842,2.0
21025,NaN,21026,940,3.0
22062,NaN,22063,989,2.0


In [12]:
cleaned_train_data.isnull().any(axis=0)

Phrase         True
PhraseId      False
SentenceId    False
Sentiment      True
dtype: bool

In [13]:
cleaned_train_data.dropna(inplace=True)
cleaned_train_data.reset_index(drop=True,inplace=True)
cleaned_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22067 entries, 0 to 22066
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Phrase      22067 non-null  object 
 1   PhraseId    22067 non-null  int64  
 2   SentenceId  22067 non-null  int64  
 3   Sentiment   22067 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 689.7+ KB


In [14]:
cleaned_train_data.shape

(22067, 4)

In [15]:
%%time
print ("Cleaning the tweets...\n")
clean_data = []
for row in test.Phrase:
    clean_data.append(tweet_cleaner_updated(row))

Cleaning the tweets...



/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


CPU times: user 8.21 s, sys: 492 ms, total: 8.71 s
Wall time: 8.81 s


In [16]:
clean_df = pd.DataFrame(clean_data, columns=['Phrase'])
clean_df['PhraseId'] = test.PhraseId
clean_df['SentenceId'] = test.SentenceId

clean_df.to_csv('cleaned_test_data.csv',encoding='utf-8')

In [17]:
csv = 'cleaned_test_data.csv'
cleaned_test_data = pd.read_csv(csv,index_col=0)
cleaned_test_data

,Phrase,PhraseId,SentenceId
0,an intermittently pleasing but mostly routine ...,156061,8545
1,an intermittently pleasing but mostly routine ...,156062,8545
2,an,156063,8545
3,intermittently pleasing but mostly routine effort,156064,8545
4,intermittently pleasing but mostly routine,156065,8545
...,...,...,...
42384,improve the over the top mix,198445,10550
42385,improve,198446,10550
42386,the over the top mix,198447,10550
42387,over the top mix,198448,10550


In [18]:
cleaned_test_data[cleaned_test_data.isnull().any(axis=1)]

,Phrase,PhraseId,SentenceId
14,NaN,156075,8545
34,NaN,156095,8546
46,NaN,156107,8546
48,NaN,156109,8546
106,NaN,156167,8547
...,...,...,...
41103,NaN,197164,10488
41290,NaN,197351,10497
41305,NaN,197366,10497
42012,NaN,198073,10529


In [19]:
cleaned_test_data.dropna(inplace=True)
cleaned_test_data.reset_index(drop=True,inplace=True)
cleaned_test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42261 entries, 0 to 42260
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Phrase      42261 non-null  object
 1   PhraseId    42261 non-null  int64 
 2   SentenceId  42261 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 990.6+ KB


In [20]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_train_data['Phrase'], cleaned_train_data['Sentiment'], test_size = 0.2, random_state = 37)
print ("X_train: ", len(X_train))
print("X_test: ", len(X_test))
print("y_train: ", len(y_train))
print("y_test: ", len(y_test))

X_train:  17653
X_test:  4414
y_train:  17653
y_test:  4414


In [21]:
cv = CountVectorizer(max_features = 1500)
cv.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1500, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [22]:
X_train_cv = cv.transform(X_train)
X_train_cv

<17653x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 73546 stored elements in Compressed Sparse Row format>

In [23]:
X_test_cv = cv.transform(X_test)
X_test_cv

<4414x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 18341 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
LRC_model = log_model.fit(X_train_cv,y_train)

y_lr = log_model.predict(X_test_cv)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [25]:
print('LR Accuracy_score: ',accuracy_score(y_test,y_lr))
print('LR confusion_matrix: ', confusion_matrix(y_lr, y_test)) 
LRC_accuracy_train  = round(LRC_model.score(X_test_cv,y_test) * 100, 2)
LRC_accuracy_train

LR Accuracy_score:  0.6246035342093339
LR confusion_matrix:  [[  19   30    6    2    1]
 [  64  192  104   18    1]
 [  55  442 2154  533   62]
 [   5   25  135  328  115]
 [   0    3    5   51   64]]


62.46

In [26]:
rfc = RandomForestClassifier(n_estimators=37, random_state=252)
RFC_model = rfc.fit(X_train_cv,y_train)

y_rfc = rfc.predict(X_test_cv)

In [27]:
print('Random Forest Accuracy_score: ',accuracy_score(y_test,y_rfc))
print('Random Forest confusion_matrix: ', confusion_matrix(y_rfc, y_test)) 
RFC_accuracy_train  = round(RFC_model.score(X_test_cv,y_test) * 100, 2)
RFC_accuracy_train 

Random Forest Accuracy_score:  0.621658359764386
Random Forest confusion_matrix:  [[  39   58   21    2    1]
 [  53  246  168   28    0]
 [  48  355 2021  467   42]
 [   3   28  184  354  116]
 [   0    5   10   81   84]]


62.17